# Adicionando funções externas utilizando LangChain

LangChain é um framework para criação de aplicações de IA e naturalmente ele possui ferramentas para facilitar a criação de funções externas para serem passadas a api da OpenAI. Para a utilização do LangChain, será necessário entendermos brevemente uma outra biblioteca de Python chamada pydantic, uma bilioteca para validação de dados que facilita a construção de estruturas de dados mais robustas.

## Introduzindo pydantic

### Como criamos uma estrutura nova sem pydantic

Sem utilizar pydantic, podemos criar uma estrutura de dados utilizando funções da seguinte forma:

In [1]:
class Pessoa:
    def __init__(self, nome: str, idade: int, peso: float) -> None:
        self.nome = nome
        self.idade = idade
        self.peso = peso

Neste caso, criamos uma função que representa uma pessoa e tem os seguintes atributos: nome, idade e peso.

In [2]:
adriano = Pessoa('Adriano', 32, 68)
adriano

In [3]:
adriano.idade

32

Podemos setar os atributos com qualquer tipo de dado, pois o Python por padrão não faz checagem de tipos.

In [4]:
adriano = Pessoa('Adriano', 32, 'ashdbadgvuya')
adriano

In [5]:
adriano.peso

'ashdbadgvuya'

### Como criamos uma estrutura nova usando pydantic

A sintaxe de pydantic acaba sendo bem mais simples para a criação de classes de dados, ao compararmos com a criação de classes comuns de Python. Nela, temos que cuidar com a definição do tipo de cada atributo, pois eles serão utilizados para validar se os dados fornecidos estão corretos.

In [6]:
from pydantic import BaseModel

class pydPessoa(BaseModel):
    nome: str
    idade: int
    peso: float

In [7]:
adriano = pydPessoa(nome='Adriano', idade=32, peso=68)
adriano

pydPessoa(nome='Adriano', idade=32, peso=68.0)

In [8]:
adriano.nome

'Adriano'

O interessante é vermos que pydantic fornece uma validação automática de dados. Isso garante uma integridade muito maior em aplicações mais complexas.

In [9]:
adriano = pydPessoa(nome='Adriano', idade=32, peso='asuhdauishg')


ValidationError: 1 validation error for pydPessoa
peso
  Input should be a valid number, unable to parse string as a number [type=float_parsing, input_value='asuhdauishg', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/float_parsing

In [10]:
adriano = pydPessoa(nome='Adriano', idade=32, peso='68')
adriano.peso

68.0

Podemos fazer um nesting de classes de pydantic, onde uma classe de dados recebe como input outra classe de pydantic.

In [11]:
from typing import List

class pydAsimoTeam(BaseModel):
    funcionarios: List[pydPessoa]

pydAsimoTeam(funcionarios=[pydPessoa(nome='Adriano', idade=32, peso=68)])

pydAsimoTeam(funcionarios=[pydPessoa(nome='Adriano', idade=32, peso=68.0)])

E a validação continua fucnionando mesmo com esta estrutura de nesting.

In [12]:
pydAsimoTeam(funcionarios=[Pessoa(nome='Adriano', idade=32, peso=68)])

ValidationError: 1 validation error for pydAsimoTeam
funcionarios.0
  Input should be a valid dictionary or instance of pydPessoa [type=model_type, input_value=<__main__.Pessoa object at 0x7f126b89f190>, input_type=Pessoa]
    For further information visit https://errors.pydantic.dev/2.10/v/model_type

## Utilizando pydantic para criação de tools da OpenAI

In [13]:
import json

def obter_temperatura_atual(local, unidade="celsius"):
    if "são paulo" in local.lower():
        return json.dumps(
            {"local": "São Paulo", "temperatura": "32", "unidade": unidade}
            )
    elif "porto alegre" in local.lower():
        return json.dumps(
            {"local": "Porto Alegre", "temperatura": "25", "unidade": unidade}
            )
    else:
        return json.dumps(
            {"local": local, "temperatura": "unknown"}
            )
    
tools = [
    {
        "type": "function",
        "function": {
            "name": "obter_temperatura_atual",
            "description": "Obtém a temperatura atual em uma dada cidade",
            "parameters": {
                "type": "object",
                "properties": {
                    "local": {
                        "type": "string",
                        "description": "O nome da cidade. Ex: São Paulo",
                    },
                    "unidade": {
                        "type": "string", 
                        "enum": ["celsius", "fahrenheit"]
                    },
                },
                "required": ["local"],
            },
        },
    }
    ]


In [14]:
from pydantic import BaseModel, Field #Importação atualizada
from typing import Optional
from enum import Enum

class UnidadeEnum(str, Enum):
    celsius = 'celsius'
    fahrenheit = 'fahrenheit'

class ObterTemperaturaAtual(BaseModel):
    """Obtém a temperatura atual de uma determinada localidade"""
    local: str = Field(description='O nome da cidade', examples=['São Paulo', 'Porto Alegre'])
    unidade: Optional[UnidadeEnum]

In [15]:
from langchain_core.utils.function_calling import convert_to_openai_function

tool_temperatura = convert_to_openai_function(ObterTemperaturaAtual)
tool_temperatura

{'name': 'ObterTemperaturaAtual',
 'description': 'Obtém a temperatura atual de uma determinada localidade',
 'parameters': {'properties': {'local': {'description': 'O nome da cidade',
    'examples': ['São Paulo', 'Porto Alegre'],
    'type': 'string'},
   'unidade': {'anyOf': [{'enum': ['celsius', 'fahrenheit'],
      'title': 'UnidadeEnum',
      'type': 'string'},
     {'type': 'null'}]}},
  'required': ['local', 'unidade'],
  'type': 'object'}}

## Adicionando função externa utilizando LangChain 

Agora que já sabemos criar funções que os modelos de llm entendam, podemos passar essas funções para os modelos de linguagem através da biblioteca langchain. Para isso temos duas formas, podemos utilizar o parâmetro functions ao chamar o método invoke dos chat_models:

In [16]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI()

resposta = chat.invoke('Qual é a temperatura de Porto Alegre', functions=[tool_temperatura])
resposta

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"local":"Porto Alegre","unidade":"celsius"}', 'name': 'ObterTemperaturaAtual'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 111, 'total_tokens': 141, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-8ee061cf-08be-4db1-8bb2-3940dc754840-0', usage_metadata={'input_tokens': 111, 'output_tokens': 30, 'total_tokens': 141, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Ou podemos dar um bind e criar um novo componente de chat_model que terá acesso a função sempre que for chamado o invoke. Nestes dois casos, o modelo se comportará com o parâmetro "auto" de chamamento de função, ou seja, ele chamará a função quando necessitar, caso contrário se comportará como um modelo de linguagem normal.

In [17]:
chat = ChatOpenAI()
chat_com_func = chat.bind(functions=[tool_temperatura])
resposta = chat_com_func.invoke('Qual é a temperatura de Porto Alegre')
resposta

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"local":"Porto Alegre","unidade":"celsius"}', 'name': 'ObterTemperaturaAtual'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 111, 'total_tokens': 141, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-755a21b4-f600-4328-aae4-d84ec42ac109-0', usage_metadata={'input_tokens': 111, 'output_tokens': 30, 'total_tokens': 141, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Podemos obrigar o modelo a sempre chamar uma função da seguinte forma:

In [18]:
resposta = chat.invoke(
    'Qual é a temperatura de Porto Alegre', 
    functions=[tool_temperatura],
    function_call={'name': 'ObterTemperaturaAtual'}
    )
resposta

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"local":"Porto Alegre","unidade":"celsius"}', 'name': 'ObterTemperaturaAtual'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 125, 'total_tokens': 141, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d710b3d7-f97f-4072-b858-56d40ed4ddd1-0', usage_metadata={'input_tokens': 125, 'output_tokens': 16, 'total_tokens': 141, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [19]:
resposta = chat.invoke(
    'Olá', 
    functions=[tool_temperatura],
    function_call={'name': 'ObterTemperaturaAtual'}
    )
resposta

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"local":"São Paulo","unidade":"celsius"}', 'name': 'ObterTemperaturaAtual'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 118, 'total_tokens': 132, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8a63144b-56c5-4ae5-a52d-334bc69c98fc-0', usage_metadata={'input_tokens': 118, 'output_tokens': 14, 'total_tokens': 132, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Adicionando a uma chain

Podemos adicionar agora este modelo com funções a um prompt e criar uma chain.

In [20]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ('system', 'Você é um assistente amigável chamado Isaac'),
    ('user', '{input}')
])

chain = prompt | chat.bind(functions=[tool_temperatura])

In [21]:
chain.invoke({'input': 'Olá'})

AIMessage(content='Olá! Como posso te ajudar hoje?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 116, 'total_tokens': 129, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-65697250-104a-47f9-a96e-4e7c527923a1-0', usage_metadata={'input_tokens': 116, 'output_tokens': 13, 'total_tokens': 129, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [22]:
chain.invoke({'input': 'Qual a temperatura em Floripa?'})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"local":"Florianópolis","unidade":"celsius"}', 'name': 'ObterTemperaturaAtual'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 121, 'total_tokens': 151, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-7c046b95-b1a9-4692-bd97-3ef8b14b62ba-0', usage_metadata={'input_tokens': 121, 'output_tokens': 30, 'total_tokens': 151, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})